In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
from requests.auth import HTTPBasicAuth
import time
%matplotlib inline

In [2]:
NUM_ITERATIONS = 10
def avg(l):
    return float(sum(l))/(len(l))

In [78]:
query0_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT DISTINCT ?vav
WHERE {
    ?vav rdf:type brick:VAV .
}
"""

query0_hod = """
SELECT ?vav
WHERE {
    ?vav rdf:type brick:VAV .
};
"""

query0_rdf3x = query0_sparql

query1_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT DISTINCT ?sensor ?room
WHERE {

    ?sensor rdf:type/rdfs:subClassOf* brick:Zone_Temperature_Sensor .
    ?room rdf:type brick:Room .
    ?vav rdf:type brick:VAV .
    ?zone rdf:type brick:HVAC_Zone .

    ?vav bf:feeds+ ?zone .
    ?zone bf:hasPart ?room .

    {?sensor bf:isPointOf ?vav }
    UNION
    {?sensor bf:isPointOf ?room }
}
"""
query1_hod = """
SELECT ?sensor ?room
WHERE {
    ?sensor rdf:type/rdfs:subClassOf* brick:Zone_Temperature_Sensor .
    ?room rdf:type brick:Room .
    ?vav rdf:type brick:VAV .
    ?zone rdf:type brick:HVAC_Zone .

    ?vav bf:feeds+ ?zone .
    ?zone bf:hasPart ?room .

    { ?sensor bf:isPointOf ?vav .
    OR
    ?sensor bf:isPointOf ?room . }
};
"""
query1_rdf3x = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT DISTINCT ?sensor ?room
WHERE {
     ?sensor rdf:type brick:ZoneTemperatureSensor .
     ?room rdf:type brick:Room .
     ?vav rdf:type brick:VAV .
     ?zone rdf:type brick:HVACZone .
     ?vav bf:feeds ?zone .
     ?zone bf:hasPart ?room .
     ?vav bf:hasPoint ?sensor .
}
"""

query2_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT ?vlv_cmd ?vav
WHERE {
    {
      { ?vlv_cmd rdf:type brick:Reheat_Valve_Command }
      UNION
      { ?vlv_cmd rdf:type brick:Cooling_Valve_Command }
    }
    ?vav rdf:type brick:VAV .
    ?vav bf:hasPoint+ ?vlv_cmd .
}
"""
query2_hod = """
SELECT ?vlv_cmd ?vav
WHERE {
    {
      { 
      ?vlv_cmd rdf:type brick:Reheat_Valve_Command .
      OR
      ?vlv_cmd rdf:type brick:Cooling_Valve_Command .
      }
    }
    ?vav rdf:type brick:VAV .
    ?vav bf:hasPoint+ ?vlv_cmd .
};
"""
query2_rdf3x = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT ?vlvcmd ?vav
WHERE {
    ?vlvcmd rdf:type brick:ReheatValveCommand .
      #UNION
      #{ ?vlvcmd rdf:type brick:CoolingValveCommand }
    ?vav rdf:type brick:VAV .
    ?vav bf:hasPoint ?vlvcmd .
}
"""

query3_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT ?floor ?room ?zone
WHERE {
    ?floor rdf:type brick:Floor .
    ?room rdf:type brick:Room .
    ?zone rdf:type brick:HVAC_Zone .

    ?room bf:isPartOf+ ?floor .
    ?room bf:isPartOf+ ?zone .
}
"""
query3_hod = """
SELECT ?floor ?room ?zone
WHERE {
    ?floor rdf:type brick:Floor .
    ?room rdf:type brick:Room .
    ?zone rdf:type brick:HVAC_Zone .

    ?room bf:isPartOf+ ?floor .
    ?room bf:isPartOf+ ?zone .
};
"""
query3_rdf3x = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT ?floor ?room ?zone
WHERE {
    ?floor rdf:type brick:Floor .
    ?room rdf:type brick:Room .
    ?zone rdf:type brick:HVACZone .
    ?room bf:isPartOf ?floor .
    ?zone bf:hasPart ?room .
}
"""

In [4]:
def run(func, show_first=False, iterations=NUM_ITERATIONS):
    query_times = []
    for i in range(iterations):
        t1 = time.time()*1000
        resp = func()
        t2 = time.time()*1000
        if resp is None: break
        if i == 0 and show_first: print resp.content
        query_times.append(t2-t1)
        print i, '{0:.2f}'.format(t2-t1)
        time.sleep(.5)
    query_avg = avg(query_times)
    print "Took {0:.2f}ms".format(query_avg)

## Hod Benchmark

In [5]:
# Hod Configs
HOD_SERVER = "http://localhost:47808/query"

In [6]:
# run hod
!cd hod; ./runserver.sh ; cd ..
#!cd hod; docker kill hod ; docker rm hod; docker run -d --name hod -p47808:47808 gtfierro/hod

+ docker pull gtfierro/hod
Using default tag: latest
latest: Pulling from gtfierro/hod
Digest: sha256:08d6d8a894838ee06f0b0dad781093b0ce827d172e89f4c3887b0ae0531bf3a7
Status: Image is up to date for gtfierro/hod:latest
+ docker kill hod
Error response from daemon: Cannot kill container hod: Container ab7ae277e28a41342ea95245b04b32614bd9a30ceb09f411f985105a6990e841 is not running
+ docker rm hod
hod
+ docker run -d --name hod -p47808:47808 gtfierro/hod
828d77b2b45f563e2c37a3a479a4da37de7c0f4e8c20cb488c85c7cf64e7eb3f


In [8]:
## query 0
def hod_query_0():
    resp = requests.post(HOD_SERVER, data=query0_hod)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp

run(hod_query_0)

0 22.07
1 11.33
2 12.07
3 6.53
4 6.63
5 7.44
6 6.79
7 7.39
8 6.87
9 6.87
Took 9.40ms


In [9]:
## query 1https://github.com/neo4j-contrib/sparql-plugin
def hod_query_1():
    resp = requests.post(HOD_SERVER, data=query1_hod)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp

run(hod_query_1)

0 88.58
1 27.01
2 26.16
3 28.50
4 22.26
5 75.20
6 35.29
7 37.77
8 37.28
9 29.26
Took 40.73ms


In [10]:
## query 2
def hod_query_2():
    resp = requests.post(HOD_SERVER, data=query2_hod)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(hod_query_2)

0 54.75
1 36.96
2 48.67
3 56.23
4 38.68
5 53.85
6 61.33
7 34.11
8 44.92
9 32.71
Took 46.22ms


In [11]:
## query 3
def hod_query_3():
    resp = requests.post(HOD_SERVER, data=query3_hod)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(hod_query_3)

0 78.20
1 61.46
2 73.69
3 61.53
4 102.88
5 104.94
6 57.38
7 62.04
8 80.03
9 65.72
Took 74.79ms


## Fuseki Benchmark

In [5]:
# Fuseki Configs
FUSEKI_SERVER = "http://localhost:3031/berkeley/query"

In [12]:
!cd fuseki; ./runserver.sh ; cd ..

+ docker pull gtfierro/fuseki
Using default tag: latest
latest: Pulling from gtfierro/fuseki
Digest: sha256:258ffe22f17a04e7792b8fedcb44f4b5b62df627958048f463d99ad26304b7d2
Status: Image is up to date for gtfierro/fuseki:latest
+ docker kill fuseki
Error response from daemon: Cannot kill container fuseki: Container ecdb3b16a2ed635f10b8a492bc2c6652ea1bb59937bb38104565219aa7974324 is not running
+ docker rm fuseki
fuseki
+ docker run -d --name fuseki -p3031:3030 gtfierro/fuseki
0a84b11090cdd546a3e112c698c8b528a7b3c37568caba1886f350855c77e714


In [7]:
## query 0
def fuseki_query_0():
    resp = requests.post(FUSEKI_SERVER, params={'query':query0_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(fuseki_query_0)

NameError: global name 'FUSEKI_SERVER' is not defined

In [ ]:
## query 1
def fuseki_query_1(iterations=1):
    resp = requests.post(FUSEKI_SERVER, params={'query':query1_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(fuseki_query_1)

In [ ]:
## query 2
def fuseki_query_2(iterations=1):
    resp = requests.post(FUSEKI_SERVER, params={'query':query2_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(fuseki_query_2, True)

## Alegrograph Benchmark

In [12]:
ALEGRO_SERVER = "http://localhost:10035/repositories/berkeley/sparql"
auth = HTTPBasicAuth('root','asdfasdf')  

In [13]:
!cd alegrograph; ./runserver.sh ; cd ..

+ docker pull gtfierro/alegrograph
Using default tag: latest
latest: Pulling from gtfierro/alegrograph












^C


In [15]:
def alegro_query_0():
    resp = requests.post(ALEGRO_SERVER, data={'query': query0_sparql}, auth=auth)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(alegro_query_0)

0 163.27
1 25.15
2 51.65
3 8.00
4 26.69
5 12.51
6 17.62
7 17.08
8 15.45
9 12.99
Took 35.04ms


In [16]:
def alegro_query_1():
    resp = requests.post(ALEGRO_SERVER, data={'query': query1_sparql}, auth=auth)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(alegro_query_1)

0 11040.61
1 4346.22
2 27305.56
3 2627.99
4 1982.31
5 1962.57


ConnectionError: HTTPConnectionPool(host='localhost', port=10035): Max retries exceeded with url: /repositories/berkeley/sparql (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x7f8b44e68810>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [ ]:
def alegro_query_2():
    resp = requests.post(ALEGRO_SERVER, data={'query': query2_sparql}, auth=auth)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(alegro_query_2)

In [ ]:
def alegro_query_3():
    resp = requests.post(ALEGRO_SERVER, data={'query': query3_sparql}, auth=auth)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(alegro_query_3)

## RDF3X Benchmark

In [29]:
RDF3X_SERVER = "http://localhost:8080/bar"

In [17]:
!cd rdf3x; ./runserver.sh ; cd ..

+ docker pull gtfierro/rdf3x
Using default tag: latest
latest: Pulling from gtfierro/rdf3x
Digest: sha256:d2e7690aaf506da2a85b69c4370e9809f7d3a94c53bb99ffea60ed06a29a7ca6
Status: Image is up to date for gtfierro/rdf3x:latest
+ docker kill rdf3x
rdf3x
+ docker rm rdf3x
rdf3x
+ docker run -d --name rdf3x -p8080:8080 gtfierro/rdf3x
5374b0e1a94ad06cf7720ac48a17fa6251735f76de149037686de803964f277c


In [30]:
def rdf3x_query_0():
    resp = requests.post(RDF3X_SERVER, data=query0_rdf3x)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(rdf3x_query_0)

0 7.89
1 6.86
2 8.53
3 5.90
4 9.84
5 5.35
6 5.73
7 5.65
8 5.62
9 5.95
Took 6.73ms


In [64]:
def rdf3x_query_1():
    resp = requests.post(RDF3X_SERVER, data=query1_rdf3x)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(rdf3x_query_1)

0 8.31
1 8.27
2 7.44
3 7.90
4 7.43
5 16.03
6 7.73
7 16.44
8 11.66
9 15.68
Took 10.69ms


In [73]:
def rdf3x_query_2():
    resp = requests.post(RDF3X_SERVER, data=query2_rdf3x)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(rdf3x_query_2)
# multiply by 2 because of lack of UNION

0 8.01
1 7.48
2 6.09
3 14.46
4 12.23
5 6.05
6 7.42
7 17.82
8 6.14
9 6.68
Took 9.24ms


In [80]:
def rdf3x_query_3():
    resp = requests.post(RDF3X_SERVER, data=query3_rdf3x)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(rdf3x_query_3)

0 8.63
1 7.71
2 8.16
3 12.25
4 11.48
5 7.81
6 8.01
7 8.01
8 7.83
9 7.93
Took 8.78ms


## rdflib Benchmark

In [17]:
RDFLIB_SERVER = "http://localhost:8081/query"

In [18]:
!cd rdflib; ./runserver.sh ; cd ..

+ docker pull gtfierro/rdflib
Using default tag: latest
latest: Pulling from gtfierro/rdflib
Digest: sha256:45a315b24e52a74dd1bad984e79638595d9a7419c3bc0dc1be0e29f36c2cb6f1
Status: Image is up to date for gtfierro/rdflib:latest
+ docker kill rdflib
rdflib
+ docker rm rdflib
rdflib
+ docker run -d --name rdflib -p8081:8081 gtfierro/rdflib
2f40a2987a2abc65e6fd33951919730ca580ce1459b6fc4379d38ff2b3f91e2f


In [21]:
def rdflib_query_0():
    resp = requests.post(RDFLIB_SERVER, data={'query': query0_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(rdflib_query_0)

0 43.70
1 25.71
2 26.06
3 32.40
4 27.11
5 31.22
6 31.37
7 33.37
8 30.19
9 26.79
Took 30.79ms


In [22]:
def rdflib_query_1():
    resp = requests.post(RDFLIB_SERVER, data={'query': query1_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(rdflib_query_1)

0 9985.79
1 10613.69
2 10239.66
3 10721.28
4 10720.05
5 10581.81
6 10220.28
7 10356.36
8 10319.36
9 10754.18
Took 10451.25ms


In [23]:
def rdflib_query_2():
    resp = requests.post(RDFLIB_SERVER, data={'query': query2_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(rdflib_query_2)

0 4937.05
1 4963.44
2 4988.96
3 5094.11
4 4926.05
5 4955.14
6 5027.94
7 5170.92
8 5411.59
9 5072.06
Took 5054.73ms


In [24]:
def rdflib_query_3():
    resp = requests.post(RDFLIB_SERVER, data={'query': query3_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(rdflib_query_3)

0 370.93
1 350.44
2 374.74
3 360.10
4 347.40
5 439.82
6 349.06
7 354.72
8 360.06
9 363.11
Took 367.04ms
